In [2]:
# NOTEBOOK CELL — OVERWRITE the Beta automation core (orchestration + API brain + Mission Control UI)
# Creates timestamped backups before overwriting.

from __future__ import annotations
from pathlib import Path
from datetime import datetime
import re

OVERWRITE = True  # you asked to overwrite

def safe_overwrite(path: str, content: str, overwrite: bool = OVERWRITE) -> None:
    p = Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)
    if p.exists() and not overwrite:
        print(f"SKIP (exists): {p}")
        return
    if p.exists() and overwrite:
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup = p.with_suffix(p.suffix + f".bak_{ts}")
        backup.write_text(p.read_text(encoding="utf-8"), encoding="utf-8")
        print(f"BACKUP: {backup}")
    p.write_text(content, encoding="utf-8")
    print(f"WROTE: {p}")

# Ensure folders
Path("src/careeragent/api").mkdir(parents=True, exist_ok=True)
Path("src/careeragent/services").mkdir(parents=True, exist_ok=True)
Path("src/careeragent/agents").mkdir(parents=True, exist_ok=True)
Path("app/ui").mkdir(parents=True, exist_ok=True)

Path("src/careeragent/__init__.py").write_text("", encoding="utf-8")
Path("src/careeragent/api/__init__.py").write_text("", encoding="utf-8")
Path("src/careeragent/services/__init__.py").write_text("", encoding="utf-8")
Path("src/careeragent/agents/__init__.py").write_text("", encoding="utf-8")
Path("app/__init__.py").write_text("", encoding="utf-8")
Path("app/ui/__init__.py").write_text("", encoding="utf-8")

CONFIG_PY = r'''
from __future__ import annotations

from functools import lru_cache
from pathlib import Path
from typing import Optional

from pydantic_settings import BaseSettings, SettingsConfigDict


def repo_root() -> Path:
    """
    Description: Resolve repository root from within src/ package.
    Layer: L0
    Input: None
    Output: Absolute Path to repo root
    """
    return Path(__file__).resolve().parents[2]


def artifacts_root() -> Path:
    """
    Description: Canonical artifacts root (local-first).
    Layer: L0
    Input: None
    Output: Path to src/careeragent/artifacts
    """
    root = Path(__file__).resolve().parents[0] / "artifacts"
    root.mkdir(parents=True, exist_ok=True)
    return root


class Settings(BaseSettings):
    """
    Description: Central configuration loader for CareerAgent-AI.
    Layer: L0
    Input: .env (repo root) + environment variables
    Output: Strongly typed settings object
    """

    model_config = SettingsConfigDict(
        env_file=str(repo_root() / ".env"),
        env_file_encoding="utf-8",
        extra="ignore",
    )

    environment: str = "local"

    # Local LLM (Ollama)
    ollama_base_url: Optional[str] = None
    ollama_model: str = "llama3.2"

    # Search
    serper_api_key: Optional[str] = None

    # LangSmith tracing
    langsmith_api_key: Optional[str] = None
    langsmith_project: str = "careeragent-ai"

    # Twilio
    twilio_account_sid: Optional[str] = None
    twilio_auth_token: Optional[str] = None
    twilio_phone: Optional[str] = None    # FROM number
    user_phone: Optional[str] = None      # default TO number


@lru_cache(maxsize=1)
def get_settings() -> Settings:
    """
    Description: Cached settings accessor.
    Layer: L0
    Input: None
    Output: Settings
    """
    return Settings()
'''
safe_overwrite("src/careeragent/config.py", CONFIG_PY)

DB_SERVICE = r'''
from __future__ import annotations

import json
import sqlite3
from pathlib import Path
from typing import Any, Dict, Optional

from careeragent.config import artifacts_root


class SqliteStateStore:
    """
    Description: Local-first persistence of OrchestrationState using SQLite.
    Layer: L8
    Input: state JSON blobs
    Output: durable run state + status polling
    """

    def __init__(self) -> None:
        """
        Description: Initialize sqlite db under artifacts/db/.
        Layer: L0
        Input: None
        Output: SqliteStateStore
        """
        db_dir = artifacts_root() / "db"
        db_dir.mkdir(parents=True, exist_ok=True)
        self._db = db_dir / "careeragent.db"
        self._init_schema()

    def _init_schema(self) -> None:
        """
        Description: Create schema if missing.
        Layer: L0
        Input: None
        Output: SQLite tables ready
        """
        with sqlite3.connect(self._db) as con:
            con.execute(
                """
                CREATE TABLE IF NOT EXISTS runs (
                    run_id TEXT PRIMARY KEY,
                    status TEXT,
                    state_json TEXT,
                    updated_at_utc TEXT
                )
                """
            )
            con.execute(
                """
                CREATE TABLE IF NOT EXISTS actions (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    run_id TEXT,
                    action_type TEXT,
                    payload_json TEXT,
                    created_at_utc TEXT
                )
                """
            )
            con.commit()

    def upsert_state(self, *, run_id: str, status: str, state: Dict[str, Any], updated_at_utc: str) -> None:
        """
        Description: Upsert a run state.
        Layer: L8
        Input: run_id + state JSON
        Output: persistent snapshot
        """
        with sqlite3.connect(self._db) as con:
            con.execute(
                """
                INSERT INTO runs(run_id, status, state_json, updated_at_utc)
                VALUES(?,?,?,?)
                ON CONFLICT(run_id) DO UPDATE SET
                    status=excluded.status,
                    state_json=excluded.state_json,
                    updated_at_utc=excluded.updated_at_utc
                """,
                (run_id, status, json.dumps(state), updated_at_utc),
            )
            con.commit()

    def get_state(self, *, run_id: str) -> Optional[Dict[str, Any]]:
        """
        Description: Fetch persisted run state.
        Layer: L8
        Input: run_id
        Output: dict | None
        """
        with sqlite3.connect(self._db) as con:
            cur = con.execute("SELECT state_json FROM runs WHERE run_id=?", (run_id,))
            row = cur.fetchone()
            if not row:
                return None
            return json.loads(row[0])

    def insert_action(self, *, run_id: str, action_type: str, payload: Dict[str, Any], created_at_utc: str) -> None:
        """
        Description: Record user/HITL action for auditability.
        Layer: L5
        Input: action payload
        Output: persisted action log
        """
        with sqlite3.connect(self._db) as con:
            con.execute(
                "INSERT INTO actions(run_id, action_type, payload_json, created_at_utc) VALUES(?,?,?,?)",
                (run_id, action_type, json.dumps(payload), created_at_utc),
            )
            con.commit()
'''
safe_overwrite("src/careeragent/services/db_service.py", DB_SERVICE)

NOTIF_SERVICE = r'''
from __future__ import annotations

from typing import Any, Dict, Literal, Optional

from careeragent.config import get_settings
from careeragent.orchestration.state import OrchestrationState


class NotificationService:
    """
    Description: L7 notifications using Twilio SMS for critical run events.
    Layer: L7
    Input: OrchestrationState + event type
    Output: SMS send attempts logged to state.meta['notifications']
    """

    def __init__(self, *, dry_run: bool = False) -> None:
        """
        Description: Initialize service (local-first safe).
        Layer: L0
        Input: dry_run
        Output: NotificationService
        """
        self._s = get_settings()
        self._dry = bool(dry_run)

    def _ready(self) -> bool:
        """
        Description: Validate Twilio env presence.
        Layer: L0
        Input: Settings
        Output: bool
        """
        return bool(self._s.twilio_account_sid and self._s.twilio_auth_token and self._s.twilio_phone)

    def send_sms(self, *, to_phone: str, body: str) -> Dict[str, Any]:
        """
        Description: Send SMS via Twilio SDK (if installed) with safe fallback.
        Layer: L7
        Input: to_phone + body
        Output: dict result
        """
        payload = {"to": to_phone, "from": self._s.twilio_phone, "body": body}

        if self._dry or not self._ready():
            return {"sent": False, "dry_run": True, "reason": "dry_run_or_missing_twilio_config", "payload": payload}

        try:
            from twilio.rest import Client  # type: ignore
        except Exception as e:
            return {"sent": False, "dry_run": True, "reason": f"twilio_sdk_missing:{e}", "payload": payload}

        client = Client(self._s.twilio_account_sid, self._s.twilio_auth_token)
        msg = client.messages.create(to=to_phone, from_=self._s.twilio_phone, body=body)
        return {"sent": True, "sid": getattr(msg, "sid", None), "payload": payload}

    def notify(self, *, state: OrchestrationState, event: Literal["needs_human_approval", "completed", "quota_error"], extra: Optional[Dict[str, Any]] = None, to_phone: Optional[str] = None) -> Dict[str, Any]:
        """
        Description: Notify on critical run state transitions.
        Layer: L7
        Input: state + event + extra
        Output: logged result
        """
        extra = extra or {}
        to = (to_phone or self._s.user_phone or "").strip()
        if not to:
            result = {"sent": False, "dry_run": True, "reason": "missing_user_phone", "event": event}
        else:
            if event == "needs_human_approval":
                body = f"CareerAgent-AI: Run {state.run_id} needs human approval. Open dashboard to review."
            elif event == "completed":
                body = f"CareerAgent-AI: Run {state.run_id} completed successfully."
            else:
                provider = extra.get("provider", "unknown")
                body = f"CareerAgent-AI: Run {state.run_id} blocked due to API quota error ({provider})."
            result = self.send_sms(to_phone=to, body=body)

        state.meta.setdefault("notifications", [])
        state.meta["notifications"].append({"event": event, "to": to, "result": result, "extra": extra})
        state.touch()
        return result
'''
safe_overwrite("src/careeragent/services/notification_service.py", NOTIF_SERVICE)

# Main autonomous orchestration with HITL actions
ORCHESTRATION_PY = r'''
from __future__ import annotations

import json
import re
import threading
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List, Literal, Optional, Tuple
from uuid import uuid4

import httpx

from careeragent.config import artifacts_root, get_settings
from careeragent.orchestration.state import OrchestrationState
from careeragent.services.db_service import SqliteStateStore
from careeragent.services.notification_service import NotificationService
from careeragent.services.health_service import HealthService

from careeragent.agents.security_agent import SanitizeAgent
from careeragent.agents.guardrail_service import OutputGuard

from careeragent.agents.parser_agent_service import ParserAgentService, ExtractedResume
from careeragent.agents.parser_evaluator_service import ParserEvaluatorService

from careeragent.agents.matcher_agent_schema import JobDescription
from careeragent.agents.matcher_agent_service import MatcherAgentService
from careeragent.agents.matcher_evaluator_service import MatchEvaluatorService

from careeragent.agents.strategy_agent_service import StrategyAgentService
from careeragent.agents.strategy_evaluator_service import StrategyEvaluatorService

from careeragent.agents.cover_letter_service import CoverLetterService
from careeragent.agents.cover_letter_evaluator_service import CoverLetterEvaluatorService

from careeragent.agents.apply_executor_service import ApplyExecutorService
from careeragent.agents.apply_executor_evaluator_service import ApplyExecutorEvaluatorService

from careeragent.services.xai_service import XAIService
from careeragent.services.exporter import CareerDossierExporter


@dataclass(frozen=True)
class JobBoard:
    """
    Description: Job board registry entry for Serper discovery.
    Layer: L3
    Input: None
    Output: JobBoard record
    """
    name: str
    domain: str


DEFAULT_JOB_BOARDS: Tuple[JobBoard, ...] = (
    JobBoard("LinkedIn Jobs", "linkedin.com/jobs"),
    JobBoard("Indeed", "indeed.com"),
    JobBoard("Glassdoor", "glassdoor.com"),
    JobBoard("ZipRecruiter", "ziprecruiter.com"),
    JobBoard("Monster", "monster.com"),
    JobBoard("Dice", "dice.com"),
    JobBoard("Lever", "jobs.lever.co"),
    JobBoard("Greenhouse", "boards.greenhouse.io"),
)


def _run_dir(run_id: str) -> Path:
    """
    Description: Resolve run directory under artifacts/runs/<run_id>.
    Layer: L0
    Input: run_id
    Output: Path
    """
    d = artifacts_root() / "runs" / run_id
    d.mkdir(parents=True, exist_ok=True)
    return d


def _save_json(p: Path, obj: Any) -> None:
    """
    Description: Safe JSON writer.
    Layer: L0
    Input: path + obj
    Output: file write
    """
    p.parent.mkdir(parents=True, exist_ok=True)
    p.write_text(json.dumps(obj, indent=2), encoding="utf-8")


class LiveFeed:
    """
    Description: Live Agent Feed logger (UI consumes it).
    Layer: L1
    Input: state + feed event
    Output: appended events + persistence
    """

    @staticmethod
    def emit(state: OrchestrationState, *, layer: str, agent: str, message: str) -> None:
        state.meta.setdefault("live_feed", [])
        state.meta["live_feed"].append({"layer": layer, "agent": agent, "message": message})
        state.touch()


class LocalResumeExtractor:
    """
    Description: Extract resume text from PDF/TXT bytes (local-first).
    Layer: L2
    Input: filename + bytes
    Output: extracted text
    """

    @staticmethod
    def extract_text(*, filename: str, data: bytes) -> str:
        name = (filename or "").lower()
        if name.endswith(".txt"):
            return data.decode("utf-8", errors="replace")
        if name.endswith(".pdf"):
            try:
                from pypdf import PdfReader  # type: ignore
            except Exception as e:
                raise RuntimeError("Missing pypdf for PDF extraction. Install: uv add pypdf") from e
            import io
            reader = PdfReader(io.BytesIO(data))
            parts = []
            for page in reader.pages:
                parts.append(page.extract_text() or "")
            return "\n".join(parts)
        return data.decode("utf-8", errors="replace")


class OllamaClient:
    """
    Description: Local LLM client for query refinement and reasoning (optional).
    Layer: L4-L5
    Input: prompts
    Output: text responses
    """

    def __init__(self, base_url: Optional[str], model: str) -> None:
        self._base = (base_url or "").strip()
        self._model = model

    def available(self) -> bool:
        return bool(self._base)

    def generate(self, *, prompt: str) -> str:
        if not self.available():
            return ""
        url = self._base.rstrip("/") + "/api/generate"
        payload = {"model": self._model, "prompt": prompt, "stream": False}
        with httpx.Client(timeout=60.0) as client:
            r = client.post(url, json=payload)
        if r.status_code >= 400:
            return ""
        return (r.json().get("response") or "").strip()


class DiscoveryService:
    """
    Description: Serper-based discovery across job boards.
    Layer: L3
    Input: query + boards
    Output: normalized results
    """

    SERPER_URL = "https://google.serper.dev/search"

    def __init__(self, *, api_key: str, health: HealthService) -> None:
        self._key = api_key
        self._health = health

    def search_board(self, *, state: OrchestrationState, step_id: str, board: JobBoard, query: str, num: int = 10) -> List[Dict[str, Any]]:
        headers = {"X-API-KEY": self._key, "Content-Type": "application/json"}
        q = f"{query} site:{board.domain}"

        try:
            with httpx.Client(timeout=30.0) as client:
                r = client.post(self.SERPER_URL, headers=headers, json={"q": q, "num": num})
        except Exception as e:
            state.status = "api_failure"
            state.meta["run_failure_code"] = "API_FAILURE"
            state.meta["run_failure_provider"] = "serper"
            LiveFeed.emit(state, layer="L3", agent="DiscoveryService", message=f"Serper request failed: {e}")
            return []

        # Quota enforcement: 403 => blocked + UI alert (HealthService does it)
        if self._health.quota.handle_serper_response(
            state=state, step_id=step_id, status_code=r.status_code, tool_name="serper.search", error_detail=r.text[:200]
        ):
            return []

        if r.status_code >= 400:
            state.status = "api_failure"
            state.meta["run_failure_code"] = "API_FAILURE"
            state.meta["run_failure_provider"] = "serper"
            LiveFeed.emit(state, layer="L3", agent="DiscoveryService", message=f"Serper error {r.status_code}: {r.text[:140]}")
            return []

        organic = (r.json().get("organic") or [])
        out = []
        for it in organic:
            out.append(
                {
                    "title": it.get("title") or "",
                    "link": it.get("link") or "",
                    "snippet": it.get("snippet") or "",
                    "source": board.name,
                    "domain": board.domain,
                }
            )
        return out


class ScraperAgent:
    """
    Description: Best-effort job page scraper.
    Layer: L3-L4
    Input: url + fallback snippet
    Output: job text
    """

    @staticmethod
    def fetch_text(*, url: str, snippet: str) -> str:
        if not url:
            return snippet or ""
        try:
            with httpx.Client(timeout=15.0, follow_redirects=True) as client:
                r = client.get(url, headers={"User-Agent": "Mozilla/5.0"})
            if r.status_code >= 400:
                return snippet or ""
            html = r.text
            txt = re.sub(r"<(script|style)[^>]*>.*?</\1>", " ", html, flags=re.S | re.I)
            txt = re.sub(r"<[^>]+>", " ", txt)
            txt = re.sub(r"\s+", " ", txt).strip()
            return (txt[:14000] if txt else (snippet or ""))
        except Exception:
            return snippet or ""


class DiscoveryEvaluatorAgent:
    """
    Description: L5 evaluator for discovery quality (controls recursive query loop).
    Layer: L5
    Input: ranking scores
    Output: EvaluationEvent + feedback (for loop-back)
    """

    def evaluate(self, *, state: OrchestrationState, target_id: str, ranking: List[Dict[str, Any]], threshold: float, retry_count: int, max_retries: int):
        scores = [float(x.get("interview_chance_score", 0.0)) for x in ranking]
        top = max(scores) if scores else 0.0
        avg = sum(scores) / max(1, len(scores))

        # Score is a confidence metric for the discovery itself (not the job scores)
        discovery_quality = min(1.0, (0.55 * top) + (0.45 * avg))
        fb: List[str] = []
        if len(ranking) < 8:
            fb.append("Too few valid jobs. Widen query/location or broaden boards.")
            discovery_quality -= 0.15
        if top < threshold:
            fb.append(f"Low confidence: top score {top*100:.1f}% below threshold {threshold*100:.0f}%.")
            fb.append("Refine query using top resume skills + role synonyms + remove noise terms.")
            discovery_quality -= 0.10

        discovery_quality = max(0.0, min(1.0, discovery_quality))

        ev = state.record_evaluation(
            layer_id="L5",
            target_id=target_id,
            generator_agent="DiscoveryAgent",
            evaluator_agent="DiscoveryEvaluatorAgent",
            evaluation_score=float(discovery_quality),
            threshold=float(threshold),
            feedback=fb,
            retry_count=int(retry_count),
            max_retries=int(max_retries),
            interview_chance=None,
        )
        return ev, top, avg


class OneClickAutomationEngine:
    """
    Description: Production-ready autonomous one-click engine with HITL gates.
    Layer: L0-L9
    Input: resume upload + preferences
    Output: stateful run with artifacts + approvals + downloadable dossier
    """

    def __init__(self) -> None:
        self._settings = get_settings()
        self._store = SqliteStateStore()
        self._health = HealthService()
        self._health.load_env(dotenv_path=str(Path(".env")))
        self._health.enable_langsmith_tracing(project=self._settings.langsmith_project)

        self._notifier = NotificationService(dry_run=not bool(self._settings.twilio_account_sid))
        self._sanitize = SanitizeAgent()
        self._out_guard = OutputGuard()

        self._ollama = OllamaClient(self._settings.ollama_base_url, self._settings.ollama_model)

        self._parser = ParserAgentService()
        self._parser_eval = ParserEvaluatorService()

        self._matcher = MatcherAgentService()
        self._matcher_eval = MatchEvaluatorService()

        self._strategist = StrategyAgentService()
        self._strategy_eval = StrategyEvaluatorService()

        self._cover = CoverLetterService()
        self._cover_eval = CoverLetterEvaluatorService()

        self._apply = ApplyExecutorService()
        self._apply_eval = ApplyExecutorEvaluatorService()

        self._disc_eval = DiscoveryEvaluatorAgent()

    # ---------- persistence ----------
    def _persist(self, state: OrchestrationState) -> None:
        d = _run_dir(state.run_id)
        _save_json(d / "state.json", state.model_dump())
        self._store.upsert_state(run_id=state.run_id, status=state.status, state=state.model_dump(), updated_at_utc=state.updated_at_utc)

    def load(self, run_id: str) -> Optional[Dict[str, Any]]:
        return self._store.get_state(run_id=run_id)

    # ---------- run lifecycle ----------
    def start_run(
        self,
        *,
        filename: str,
        data: bytes,
        prefs: Dict[str, Any],
    ) -> OrchestrationState:
        state = OrchestrationState.new(env=self._settings.environment, mode="agentic", git_sha=None)
        state.meta["preferences"] = prefs
        state.meta.setdefault("job_scores", {})
        state.meta.setdefault("job_components", {})
        state.meta.setdefault("job_meta", {})

        LiveFeed.emit(state, layer="L1", agent="Dashboard", message="Run created. Starting autonomous pipeline…")
        self._persist(state)

        # run in background thread (local-first, UI polls)
        t = threading.Thread(target=self._run_autonomous, args=(state.run_id, filename, data), daemon=True)
        t.start()
        return state

    def submit_action(self, *, run_id: str, action_type: str, payload: Dict[str, Any]) -> OrchestrationState:
        raw = self.load(run_id)
        if not raw:
            raise ValueError("run_id not found")
        state = OrchestrationState(**raw)

        # audit action
        self._store.insert_action(run_id=run_id, action_type=action_type, payload=payload, created_at_utc=state.updated_at_utc)

        LiveFeed.emit(state, layer="L5", agent="HITL", message=f"User action received: {action_type}")
        state.meta["last_user_action"] = {"type": action_type, "payload": payload}
        self._persist(state)

        # Continue in background
        t = threading.Thread(target=self._continue_after_hitl, args=(run_id,), daemon=True)
        t.start()
        return state

    # ---------- autonomous core ----------
    def _run_autonomous(self, run_id: str, filename: str, data: bytes) -> None:
        raw = self.load(run_id)
        if not raw:
            return
        state = OrchestrationState(**raw)
        run_dir = _run_dir(run_id)

        # L2: Extract text from file upload
        state.start_step("l2_resume_extract", layer_id="L2", tool_name="ResumeExtractor", input_ref={"filename": filename})
        try:
            text = LocalResumeExtractor.extract_text(filename=filename, data=data)
        except Exception as e:
            state.end_step("l2_resume_extract", status="failed", output_ref={}, message=str(e))
            state.status = "needs_human_approval"
            state.meta["pending_action"] = "resume_extract_failed"
            LiveFeed.emit(state, layer="L2", agent="ParserAgent", message=f"Resume extraction failed: {e}")
            self._persist(state)
            self._notify(state, event="needs_human_approval")
            return

        (run_dir / "resume_raw.txt").write_text(text, encoding="utf-8")
        state.add_artifact("resume_raw", str(run_dir / "resume_raw.txt"), content_type="text/plain")
        state.end_step("l2_resume_extract", status="ok", output_ref={"artifact_key": "resume_raw"}, message="extracted")
        LiveFeed.emit(state, layer="L2", agent="ParserAgent", message="Resume extracted from upload.")
        self._persist(state)

        # L0: Sanitize
        state.start_step("l0_sanitize", layer_id="L0", tool_name="SanitizeAgent", input_ref={})
        safe = self._sanitize.sanitize_before_llm(state=state, step_id="l0_sanitize", tool_name="sanitize_before_llm", user_text=text, context="resume")
        if safe is None:
            LiveFeed.emit(state, layer="L0", agent="SanitizeAgent", message="Prompt injection detected. Run blocked.")
            self._persist(state)
            self._notify(state, event="quota_error", extra={"provider": "security_block"})
            return
        state.end_step("l0_sanitize", status="ok", output_ref={"sanitized": True}, message="pass")
        LiveFeed.emit(state, layer="L0", agent="SanitizeAgent", message="Security check passed.")
        self._persist(state)

        # L2/L3: Parse + eval loop
        extracted = self._parse_with_gate(state=state, safe_text=safe, run_dir=run_dir)
        self._persist(state)
        if state.status == "needs_human_approval":
            self._notify(state, event="needs_human_approval")
            return

        # L3-L5: Autonomous job hunt with recursive refinement
        self._hunt_loop(state=state, extracted=extracted, run_dir=run_dir)
        self._persist(state)
        if state.status == "needs_human_approval":
            self._notify(state, event="needs_human_approval")
            return

    def _continue_after_hitl(self, run_id: str) -> None:
        raw = self.load(run_id)
        if not raw:
            return
        state = OrchestrationState(**raw)
        run_dir = _run_dir(run_id)

        pending = state.meta.get("pending_action")
        action = (state.meta.get("last_user_action") or {}).get("type")
        payload = (state.meta.get("last_user_action") or {}).get("payload") or {}

        # If user rejected ranking => restart discovery with refined preference notes
        if pending == "review_ranking" and action == "reject_ranking":
            state.status = "running"
            state.meta["pending_action"] = None
            state.meta.setdefault("user_refinement_notes", [])
            state.meta["user_refinement_notes"].append(payload.get("reason", "User rejected ranking"))
            LiveFeed.emit(state, layer="L5", agent="HITL", message="Ranking rejected. Re-running discovery loop with refined query…")
            self._persist(state)

            # Reload extracted resume
            ex_ref = state.artifacts.get("extracted_resume")
            if not ex_ref:
                state.status = "needs_human_approval"
                state.meta["pending_action"] = "missing_extracted_resume"
                self._persist(state)
                return
            extracted = ExtractedResume(**json.loads(Path(ex_ref.path).read_text(encoding="utf-8")))
            self._hunt_loop(state=state, extracted=extracted, run_dir=run_dir)
            self._persist(state)
            if state.status == "needs_human_approval":
                self._notify(state, event="needs_human_approval")
            return

        # Approve ranking => generate strategy + cover letters + eval + guardrails, then pause for draft review
        if pending == "review_ranking" and action == "approve_ranking":
            LiveFeed.emit(state, layer="L6", agent="Generator", message="Ranking approved. Generating strategies + cover letters…")
            self._persist(state)

            ranking_path = Path(run_dir / "ranking.json")
            if not ranking_path.exists():
                state.status = "needs_human_approval"
                state.meta["pending_action"] = "ranking_missing"
                self._persist(state)
                return
            ranking = json.loads(ranking_path.read_text(encoding="utf-8"))

            self._generate_drafts_bundle(state=state, extracted=None, ranking=ranking, run_dir=run_dir)
            state.status = "needs_human_approval"
            state.meta["pending_action"] = "review_drafts"
            LiveFeed.emit(state, layer="L5", agent="HITL", message="Drafts ready for review. Approve to submit (simulated).")
            self._persist(state)
            self._notify(state, event="needs_human_approval")
            return

        # Approve drafts => apply + XAI + dossier => completed
        if pending == "review_drafts" and action == "approve_drafts":
            LiveFeed.emit(state, layer="L7", agent="ApplyExecutor", message="Drafts approved. Submitting applications (simulated)…")
            self._persist(state)
            self._apply_and_finalize(state=state, run_dir=run_dir)
            self._persist(state)
            if state.status == "completed":
                self._notify(state, event="completed")
            return

        # Reject drafts => return to review_ranking (or rerun generation)
        if pending == "review_drafts" and action == "reject_drafts":
            state.status = "needs_human_approval"
            state.meta["pending_action"] = "review_ranking"
            state.meta.setdefault("user_refinement_notes", [])
            state.meta["user_refinement_notes"].append(payload.get("reason", "User rejected drafts"))
            LiveFeed.emit(state, layer="L5", agent="HITL", message="Drafts rejected. Returning to ranking review / refinement.")
            self._persist(state)
            self._notify(state, event="needs_human_approval")
            return

    # ---------- helpers ----------
    def _notify(self, state: OrchestrationState, *, event: Literal["needs_human_approval", "completed", "quota_error"], extra: Optional[Dict[str, Any]] = None) -> None:
        to = (state.meta.get("preferences", {}) or {}).get("user_phone") or self._settings.user_phone
        if to:
            self._notifier.notify(state=state, event=event, extra=extra, to_phone=str(to))

    def _parse_with_gate(self, *, state: OrchestrationState, safe_text: str, run_dir: Path) -> ExtractedResume:
        feedback: List[str] = []
        extracted: Optional[ExtractedResume] = None

        for attempt in range(0, 4):
            sid = f"l2_parse_{attempt+1}"
            state.start_step(sid, layer_id="L2", tool_name="ParserAgentService", input_ref={"attempt": attempt+1})
            extracted = self._parser.parse(raw_text=safe_text, orchestration_state=state, feedback=feedback)
            p = run_dir / f"extracted_resume_attempt_{attempt+1}.json"
            _save_json(p, extracted.to_json_dict())
            state.add_artifact(f"extracted_resume_attempt_{attempt+1}", str(p), content_type="application/json")
            state.end_step(sid, status="ok", output_ref={"artifact_key": f"extracted_resume_attempt_{attempt+1}"}, message="parsed")

            ev = self._parser_eval.evaluate(
                orchestration_state=state,
                raw_text=safe_text,
                extracted=extracted,
                target_id="resume_main",
                threshold=0.80,
                retry_count=attempt,
                max_retries=3,
            )
            decision = state.apply_recursive_gate(target_id="resume_main", layer_id="L3")
            LiveFeed.emit(state, layer="L3", agent="ParserEvaluator", message=f"Parse score={ev.evaluation_score:.2f} decision={decision}")

            if decision == "pass":
                state.add_artifact("extracted_resume", str(p), content_type="application/json")
                LiveFeed.emit(state, layer="L2", agent="ParserAgent", message="Profile extracted successfully.")
                return extracted

            if decision == "human_approval":
                state.status = "needs_human_approval"
                state.meta["pending_action"] = "resume_cleanup"
                LiveFeed.emit(state, layer="L3", agent="ParserEvaluator", message="Low confidence parsing. Needs human cleanup.")
                return extracted

            feedback = ev.feedback

        state.status = "needs_human_approval"
        state.meta["pending_action"] = "resume_cleanup"
        return extracted or ExtractedResume()

    def _hunt_loop(self, *, state: OrchestrationState, extracted: ExtractedResume, run_dir: Path) -> None:
        prefs = state.meta.get("preferences", {}) or {}
        target_role = str(prefs.get("target_role", "Data Scientist"))
        country = str(prefs.get("country", "US"))
        location = str(prefs.get("location", "United States"))
        remote = bool(prefs.get("remote", True))
        salary = str(prefs.get("salary", ""))

        # weights
        state.meta["w1_skill_overlap"] = 0.45
        state.meta["w2_experience_alignment"] = 0.35
        state.meta["w3_ats_score"] = 0.20

        # Discovery ready?
        if not self._settings.serper_api_key:
            state.status = "needs_human_approval"
            state.meta["pending_action"] = "missing_serper_key"
            LiveFeed.emit(state, layer="L3", agent="DiscoveryService", message="Missing SERPER_API_KEY. Add it to .env.")
            return

        discovery = DiscoveryService(api_key=self._settings.serper_api_key, health=self._health)
        guard_threshold = float(prefs.get("discovery_threshold", 0.70))
        max_refine = int(prefs.get("max_refinements", 3))

        # build query
        top_skills = (extracted.skills or [])[:6]
        intent = "remote" if remote else "on-site"
        salary_part = f'"{salary}"' if salary else ""
        base_query = f'{target_role} {location} {intent} {salary_part} ' + " ".join(top_skills)

        # incorporate user refinement notes
        notes = state.meta.get("user_refinement_notes", []) or []
        if notes:
            base_query += " " + " ".join([str(n)[:40] for n in notes[-2:]])

        for attempt in range(0, max_refine + 1):
            sid = f"l3_discovery_{attempt+1}"
            state.start_step(sid, layer_id="L3", tool_name="DiscoveryService", input_ref={"attempt": attempt+1, "query": base_query})
            LiveFeed.emit(state, layer="L3", agent="DiscoveryAgent", message=f"Searching across job boards… (attempt {attempt+1})")

            results = self._search_all_boards(state=state, discovery=discovery, query=base_query)
            state.end_step(sid, status="ok", output_ref={"results": len(results)}, message="discovery_done")

            if state.status in ("blocked", "api_failure"):
                LiveFeed.emit(state, layer="L3", agent="DiscoveryAgent", message="Discovery halted due to API block/failure.")
                if state.status == "blocked":
                    self._notify(state, event="quota_error", extra={"provider": "serper"})
                return

            # Scrape + embed (local)
            docs = []
            LiveFeed.emit(state, layer="L4", agent="VectorAgent", message=f"Scraping + embedding {min(8,len(results))} jobs…")
            for it in results[:8]:
                s2 = f"l4_scrape_{uuid4().hex[:8]}"
                state.start_step(s2, layer_id="L4", tool_name="ScraperAgent", input_ref={"url": it.get('link'), "source": it.get("source")})
                text = ScraperAgent.fetch_text(url=it.get("link") or "", snippet=it.get("snippet") or "")
                doc_id = uuid4().hex
                meta = {"title": it.get("title"), "url": it.get("link"), "source": it.get("source")}
                docs.append({"doc_id": doc_id, "text": text, "meta": meta})
                state.end_step(s2, status="ok", output_ref={"doc_id": doc_id}, message="scraped")
                LiveFeed.emit(state, layer="L4", agent="VectorAgent", message=f"Embedded: {meta['source']} | {str(meta['title'])[:60]}")

            # Match + evaluator twin consistency check (math)
            ranking = self._match_and_rank(state=state, extracted=extracted, docs=docs, target_role=target_role, country=country)
            _save_json(run_dir / "ranking.json", ranking)
            state.add_artifact("ranking", str(run_dir / "ranking.json"), content_type="application/json")

            ev, top, avg = self._disc_eval.evaluate(state=state, target_id="discovery_quality", ranking=ranking, threshold=guard_threshold, retry_count=attempt, max_retries=max_refine)
            decision = state.apply_recursive_gate(target_id="discovery_quality", layer_id="L5")

            LiveFeed.emit(state, layer="L5", agent="EvaluatorAgent",
                          message=f"Search results validated. top={top*100:.1f}% avg={avg*100:.1f}% decision={decision}")

            # HITL pause when either low confidence after retries OR when ranking ready
            if decision == "pass":
                state.status = "needs_human_approval"
                state.meta["pending_action"] = "review_ranking"
                LiveFeed.emit(state, layer="L1", agent="Dashboard", message="One-Click ranking ready for review.")
                return

            if decision == "human_approval":
                state.status = "needs_human_approval"
                state.meta["pending_action"] = "low_confidence_discovery"
                LiveFeed.emit(state, layer="L5", agent="EvaluatorAgent", message="Low confidence discovery. Needs guidance.")
                return

            # retry => refine query autonomously (LLM if available, else deterministic)
            base_query = self._refine_query(state=state, base_query=base_query, extracted=extracted, ranking=ranking, feedback=ev.feedback, target_role=target_role, location=location)
            LiveFeed.emit(state, layer="L3", agent="DiscoveryAgent", message=f"Refining search… new query: {base_query[:140]}")

    def _search_all_boards(self, *, state: OrchestrationState, discovery: DiscoveryService, query: str) -> List[Dict[str, Any]]:
        seen = set()
        out: List[Dict[str, Any]] = []
        for b in DEFAULT_JOB_BOARDS:
            step_id = f"l3_serper_{b.domain.replace('/','_')}"
            state.start_step(step_id, layer_id="L3", tool_name="serper.search", input_ref={"board": b.name})
            items = discovery.search_board(state=state, step_id=step_id, board=b, query=query, num=6)
            state.end_step(step_id, status="ok", output_ref={"count": len(items)}, message=b.name)

            for it in items:
                link = it.get("link") or ""
                if not link or link in seen:
                    continue
                seen.add(link)
                out.append(it)
        return out

    def _match_and_rank(self, *, state: OrchestrationState, extracted: ExtractedResume, docs: List[Dict[str, Any]], target_role: str, country: str) -> List[Dict[str, Any]]:
        # dictionary for quick skill extraction from job text
        dictionary = list(dict.fromkeys([(s or "").lower() for s in (extracted.skills or []) if s]))[:40]
        if not dictionary:
            dictionary = ["python", "sql", "mlflow", "docker", "kubernetes", "fastapi", "azure", "aws", "rag", "langgraph"]

        ranked: List[Dict[str, Any]] = []
        for d in docs:
            text_low = (d.get("text") or "").lower()
            req = [s for s in dictionary if s in text_low][:12]
            pref = [s for s in dictionary if s in text_low][12:18]
            meta = d.get("meta") or {}
            job = JobDescription(
                job_id=str(d.get("doc_id")),
                role_title=str(meta.get("title") or target_role),
                company=str(meta.get("source") or "unknown"),
                country_code=str(country),
                required_skills=[str(x) for x in req],
                preferred_skills=[str(x) for x in pref],
                responsibilities=[],
                requirements_text=str(meta.get("title") or "") + " " + (d.get("text") or "")[:2500],
                applicants_count=None,
                market_competition_factor=None,
            )

            s_match = f"l4_match_{uuid4().hex[:8]}"
            state.start_step(s_match, layer_id="L4", tool_name="MatcherAgent", input_ref={"job_id": job.job_id})
            report = self._matcher.match(resume=extracted, job=job, orchestration_state=state)
            state.end_step(s_match, status="ok", output_ref={"score": report.interview_chance_score}, message="matched")

            # Evaluator twin checks scoring math
            ev = self._matcher_eval.evaluate(
                orchestration_state=state,
                resume=extracted,
                job=job,
                report=report,
                target_id=f"match::{job.job_id}",
                threshold=0.80,
                retry_count=0,
                max_retries=3,
            )
            _ = state.apply_recursive_gate(target_id=f"match::{job.job_id}", layer_id="L4")

            state.meta.setdefault("job_scores", {})
            state.meta.setdefault("job_components", {})
            state.meta.setdefault("job_meta", {})
            state.meta["job_scores"][job.job_id] = float(report.interview_chance_score)
            state.meta["job_components"][job.job_id] = report.components.model_dump()
            state.meta["job_meta"][job.job_id] = {"role_title": job.role_title, "company": job.company, "url": meta.get("url"), "source": meta.get("source")}

            ranked.append(
                {
                    "job_id": job.job_id,
                    "role_title": job.role_title,
                    "company": job.company,
                    "source": meta.get("source"),
                    "url": meta.get("url"),
                    "interview_chance_score": float(report.interview_chance_score),
                    "overall_match_percent": float(report.overall_match_percent),
                    "matched_skills": report.matched_skills[:10],
                    "missing_required_skills": report.missing_required_skills[:10],
                    "evaluator_score": float(ev.evaluation_score),
                    "evaluator_feedback": (ev.feedback[:2] if ev.feedback else []),
                }
            )

        ranked.sort(key=lambda x: x["interview_chance_score"], reverse=True)
        return ranked

    def _refine_query(self, *, state: OrchestrationState, base_query: str, extracted: ExtractedResume, ranking: List[Dict[str, Any]], feedback: List[str], target_role: str, location: str) -> str:
        top = ranking[0] if ranking else {}
        top_skills = (top.get("matched_skills") or [])[:5]
        resume_skills = (extracted.skills or [])[:5]
        hints = " ".join(list(dict.fromkeys([*top_skills, *resume_skills]))[:8])

        # LLM refinement if available and safe
        if self._ollama.available():
            prompt = (
                "Refine a job search query for Serper.\n"
                f"Target role: {target_role}\nLocation: {location}\n"
                f"Current query: {base_query}\n"
                f"Resume skills hint: {hints}\n"
                f"Evaluator feedback: {feedback}\n"
                "Return ONE improved concise query string only."
            )
            # sanitize prompt injection risk (treat as user-provided)
            sid = f"l0_refine_sanitize_{uuid4().hex[:6]}"
            state.start_step(sid, layer_id="L0", tool_name="SanitizeAgent", input_ref={"context": "query_refine"})
            safe = self._sanitize.sanitize_before_llm(state=state, step_id=sid, tool_name="sanitize_before_llm", user_text=prompt, context="chat")
            if safe is None:
                state.status = "needs_human_approval"
                state.meta["pending_action"] = "query_refine_blocked"
                return base_query
            state.end_step(sid, status="ok", output_ref={}, message="ok")

            out = self._ollama.generate(prompt=safe)
            if out:
                return out.strip().replace("\n", " ")[:220]

        # deterministic fallback
        return f'{target_role} {location} {hints} "apply"'

    def _generate_drafts_bundle(self, *, state: OrchestrationState, extracted: Optional[ExtractedResume], ranking: List[Dict[str, Any]], run_dir: Path) -> None:
        # load extracted if not provided
        if extracted is None:
            ex_ref = state.artifacts.get("extracted_resume")
            if ex_ref and Path(ex_ref.path).exists():
                extracted = ExtractedResume(**json.loads(Path(ex_ref.path).read_text(encoding="utf-8")))
            else:
                state.status = "needs_human_approval"
                state.meta["pending_action"] = "missing_extracted_resume"
                return

        drafts: List[Dict[str, Any]] = []
        strategies: List[Dict[str, Any]] = []

        top_jobs = ranking[:5]
        for j in top_jobs:
            # Build minimal JobDescription from meta for strategy/cover
            job = JobDescription(
                job_id=str(j["job_id"]),
                role_title=str(j.get("role_title") or "Role"),
                company=str(j.get("company") or "Company"),
                country_code="US",
                required_skills=[],
                preferred_skills=[],
                responsibilities=[],
                requirements_text="",
                applicants_count=None,
                market_competition_factor=None,
            )

            # strategy
            s_id = f"l5_strategy_{uuid4().hex[:6]}"
            state.start_step(s_id, layer_id="L5", tool_name="StrategyAgent", input_ref={"job_id": job.job_id})
            # we don't have full MatchReport object here without re-running; keep it lightweight:
            # create a pseudo MatchReport by reading components already in meta (optional)
            # For now, strategy is optional; if missing, still proceed.
            try:
                # reconstruct a small match report file if exists, else skip
                # (keeps this robust even if job boards block scraping)
                from careeragent.agents.matcher_agent_schema import MatchReport, MatchComponents
                comps = state.meta.get("job_components", {}).get(job.job_id, {})
                match_report = MatchReport(
                    job_id=job.job_id,
                    role_title=job.role_title,
                    company=job.company,
                    matched_skills=j.get("matched_skills") or [],
                    missing_required_skills=j.get("missing_required_skills") or [],
                    missing_preferred_skills=[],
                    components=MatchComponents(**comps) if comps else MatchComponents(skill_overlap=0.0, experience_alignment=0.0, ats_score=0.0, market_competition_factor=1.0),
                    interview_chance_score=float(j.get("interview_chance_score", 0.0)),
                    overall_match_percent=float(j.get("overall_match_percent", 0.0)),
                    rationale=[],
                )
                strat = self._strategist.generate(resume=extracted, job=job, match_report=match_report, orchestration_state=state, feedback=[])
                strategies.append(strat.model_dump())
            except Exception:
                strategies.append({"job_id": job.job_id, "note": "strategy generation skipped (insufficient match object)."})
            state.end_step(s_id, status="ok", output_ref={}, message="strategy_done")

            # cover letter + evaluator + output guard
            c_id = f"l6_cover_{uuid4().hex[:6]}"
            state.start_step(c_id, layer_id="L6", tool_name="CoverLetterService", input_ref={"job_id": job.job_id})
            try:
                from careeragent.agents.matcher_agent_schema import MatchReport, MatchComponents
                comps = state.meta.get("job_components", {}).get(job.job_id, {})
                match_report = MatchReport(
                    job_id=job.job_id,
                    role_title=job.role_title,
                    company=job.company,
                    matched_skills=j.get("matched_skills") or [],
                    missing_required_skills=j.get("missing_required_skills") or [],
                    missing_preferred_skills=[],
                    components=MatchComponents(**comps) if comps else MatchComponents(skill_overlap=0.0, experience_alignment=0.0, ats_score=0.0, market_competition_factor=1.0),
                    interview_chance_score=float(j.get("interview_chance_score", 0.0)),
                    overall_match_percent=float(j.get("overall_match_percent", 0.0)),
                    rationale=[],
                )
                draft = self._cover.draft(resume=extracted, job=job, match_report=match_report, orchestration_state=state, feedback=["Include contact header"])
                # evaluator twin
                ev = self._cover_eval.evaluate(orchestration_state=state, resume=extracted, job=job, match_report=match_report, draft=draft, target_id=f"cover::{job.job_id}", threshold=0.80, retry_count=0, max_retries=3)
                _ = state.apply_recursive_gate(target_id=f"cover::{job.job_id}", layer_id="L6")

                # output guard
                guard = self._out_guard.check_cover_letter(state=state, draft_text=draft.body, resume=extracted, match_report=match_report, country_code="US")
                if guard.action == "needs_revision":
                    LiveFeed.emit(state, layer="L0", agent="OutputGuard", message=f"Cover letter flagged for {job.job_id}: {guard.issues[:1]}")
                # persist
                p = run_dir / f"cover_letter_{job.job_id}.md"
                p.write_text(draft.body, encoding="utf-8")
                state.add_artifact(f"cover_letter_{job.job_id}", str(p), content_type="text/markdown")
                drafts.append({"job_id": job.job_id, "path": str(p), "evaluator_score": float(ev.evaluation_score), "guard_action": guard.action, "guard_issues": guard.issues[:3]})
            except Exception as e:
                drafts.append({"job_id": job.job_id, "error": str(e)})
            state.end_step(c_id, status="ok", output_ref={}, message="cover_done")

        _save_json(run_dir / "drafts.json", {"drafts": drafts, "strategies": strategies})
        state.add_artifact("drafts_bundle", str(run_dir / "drafts.json"), content_type="application/json")
        LiveFeed.emit(state, layer="L6", agent="Generator", message=f"Generated drafts for {len(drafts)} jobs.")

    def _apply_and_finalize(self, *, state: OrchestrationState, run_dir: Path) -> None:
        # Apply to top 3 cover letters that exist
        ranking_path = run_dir / "ranking.json"
        if not ranking_path.exists():
            state.status = "needs_human_approval"
            state.meta["pending_action"] = "ranking_missing"
            return

        ranking = json.loads(ranking_path.read_text(encoding="utf-8"))[:3]
        submissions = []
        for j in ranking:
            job_id = str(j["job_id"])
            cover_key = f"cover_letter_{job_id}"
            if cover_key not in state.artifacts:
                continue
            s_id = f"l7_apply_{uuid4().hex[:6]}"
            state.start_step(s_id, layer_id="L7", tool_name="ApplyExecutorService", input_ref={"job_id": job_id})
            sub = self._apply.submit(orchestration_state=state, job_id=job_id, resume_artifact_key="extracted_resume", cover_letter_artifact_key=cover_key, notes="Simulated submit (beta).")
            submissions.append(sub.model_dump())
            state.end_step(s_id, status="ok", output_ref={"submission_id": sub.submission_id}, message="submitted")

            ev = self._apply_eval.evaluate(orchestration_state=state, submission=sub, target_id=f"apply::{sub.submission_id}", threshold=0.90, retry_count=0, max_retries=3)
            _ = state.apply_recursive_gate(target_id=f"apply::{sub.submission_id}", layer_id="L7")

        _save_json(run_dir / "submissions.json", {"submissions": submissions})
        state.add_artifact("submissions", str(run_dir / "submissions.json"), content_type="application/json")

        # XAI PDF + dossier
        xai = XAIService()
        xai_paths = xai.write_outputs(state=state, require_reportlab=False)
        state.add_artifact("xai_transparency_pdf", xai_paths["pdf"], content_type="application/pdf")
        state.add_artifact("transparency_matrix_json", xai_paths["json"], content_type="application/json")

        exporter = CareerDossierExporter()
        bundle = exporter.bundle_reports(run_id=state.run_id, final_pdf_path=xai_paths["pdf"])
        state.add_artifact("career_dossier_zip", bundle["zip"], content_type="application/zip")

        state.status = "completed"
        LiveFeed.emit(state, layer="L9", agent="Analytics", message="Finalized dossier (XAI PDF + ZIP). Run completed.")


# Singleton engine for API usage
ENGINE = OneClickAutomationEngine()
'''
safe_overwrite("src/careeragent/orchestration.py", ORCHESTRATION_PY)

API_MAIN = r'''
from __future__ import annotations

import json
from typing import Any, Dict, Optional

from fastapi import FastAPI, File, Form, HTTPException, UploadFile
from pydantic import BaseModel, Field

from careeragent.orchestration import ENGINE


app = FastAPI(title="CareerAgent-AI Beta Brain", version="0.2.0")


class AnalyzeResponse(BaseModel):
    """
    Description: /analyze response
    Layer: L1
    Input: run result init
    Output: run_id + status
    """
    run_id: str
    status: str


class ActionRequest(BaseModel):
    """
    Description: HITL action request payload.
    Layer: L5
    Input: action_type + payload
    Output: state transition
    """
    action_type: str = Field(..., description="approve_ranking | reject_ranking | approve_drafts | reject_drafts")
    payload: Dict[str, Any] = Field(default_factory=dict)


@app.post("/analyze", response_model=AnalyzeResponse)
async def analyze(
    resume: UploadFile = File(...),
    preferences_json: str = Form(...),
) -> AnalyzeResponse:
    """
    Description: One-click analyze endpoint. Accepts resume file + preferences JSON.
    Layer: L1
    Input: multipart form
    Output: run_id
    """
    try:
        prefs = json.loads(preferences_json)
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Invalid preferences_json: {e}")

    data = await resume.read()
    st = ENGINE.start_run(filename=resume.filename, data=data, prefs=prefs)
    return AnalyzeResponse(run_id=st.run_id, status=st.status)


@app.get("/status/{run_id}")
def status(run_id: str) -> Dict[str, Any]:
    """
    Description: Poll persisted state.
    Layer: L1
    Input: run_id
    Output: state JSON
    """
    st = ENGINE.load(run_id)
    if not st:
        raise HTTPException(status_code=404, detail="run_id not found")
    return st


@app.post("/action/{run_id}")
def action(run_id: str, req: ActionRequest) -> Dict[str, Any]:
    """
    Description: Submit HITL action and continue automation.
    Layer: L5
    Input: action_type + payload
    Output: updated state JSON
    """
    try:
        st = ENGINE.submit_action(run_id=run_id, action_type=req.action_type, payload=req.payload)
    except ValueError:
        raise HTTPException(status_code=404, detail="run_id not found")
    return st.model_dump()
'''
safe_overwrite("src/careeragent/api/main.py", API_MAIN)

DASHBOARD = r'''
from __future__ import annotations

import json
from pathlib import Path
from typing import Any, Dict, Optional

import requests
import streamlit as st


def main() -> None:
    """
    Description: Mission Control dashboard (Live Agent Feed + HITL approvals + downloads).
    Layer: L1
    Input: user resume + preferences
    Output: interactive one-click automation UI
    """
    st.set_page_config(page_title="CareerAgent-AI Mission Control", layout="wide")
    st.title("CareerAgent-AI — Mission Control (One-Click Automation)")

    API_BASE = st.sidebar.text_input("API Base URL", value="http://127.0.0.1:8000")

    st.sidebar.subheader("Upload + Preferences")
    resume_file = st.sidebar.file_uploader("Resume (PDF/TXT)", type=["pdf", "txt"])
    target_role = st.sidebar.text_input("Target Role", value="Data Scientist")
    country = st.sidebar.text_input("Country", value="US")
    location = st.sidebar.text_input("Location", value="United States")
    remote = st.sidebar.checkbox("Remote preferred", value=True)
    salary = st.sidebar.text_input("Salary preference (optional)", value="")
    user_phone = st.sidebar.text_input("Phone for SMS (optional)", value="")

    st.sidebar.subheader("Loop Controls")
    discovery_threshold = st.sidebar.slider("Discovery confidence threshold", 0.50, 0.90, 0.70, 0.05)
    max_refinements = st.sidebar.slider("Max refinements", 1, 5, 3, 1)

    run_btn = st.sidebar.button("🚀 RUN ONE-CLICK", type="primary", use_container_width=True, disabled=(resume_file is None))

    st.sidebar.subheader("Existing Run")
    run_id_in = st.sidebar.text_input("Run ID", value=st.session_state.get("run_id", ""))

    def _poll(run_id: str) -> Optional[Dict[str, Any]]:
        try:
            r = requests.get(f"{API_BASE}/status/{run_id}", timeout=20)
            if r.status_code != 200:
                return None
            return r.json()
        except Exception:
            return None

    if run_btn:
        prefs = {
            "target_role": target_role,
            "country": country,
            "location": location,
            "remote": remote,
            "salary": salary,
            "user_phone": user_phone.strip() or None,
            "discovery_threshold": float(discovery_threshold),
            "max_refinements": int(max_refinements),
        }

        files = {"resume": (resume_file.name, resume_file.getvalue())}
        data = {"preferences_json": json.dumps(prefs)}
        r = requests.post(f"{API_BASE}/analyze", files=files, data=data, timeout=120)
        if r.status_code >= 400:
            st.error(f"/analyze failed: {r.status_code} {r.text[:500]}")
        else:
            out = r.json()
            st.session_state["run_id"] = out["run_id"]
            st.success(f"Run started: {out['run_id']} (status: {out['status']})")

    run_id = st.session_state.get("run_id") or run_id_in.strip()
    if not run_id:
        st.info("Upload a resume and click RUN, or paste an existing run_id in the sidebar.")
        return

    st.subheader(f"Run: {run_id}")
    colA, colB, colC = st.columns([1, 1, 1])
    with colA:
        if st.button("🔄 Refresh"):
            pass
    data = _poll(run_id)
    if not data:
        st.warning("Run not found yet. Try refresh.")
        return

    status = data.get("status", "unknown")
    meta = data.get("meta", {}) or {}
    feed = meta.get("live_feed", []) or []
    steps = data.get("steps", []) or []
    artifacts = data.get("artifacts", {}) or {}
    pending = meta.get("pending_action")

    with colA:
        st.metric("Status", status)
    with colB:
        st.metric("Pending Action", str(pending))
    with colC:
        js = (meta.get("job_scores") or {})
        if js:
            job_id, score = next(iter(js.items()))
            st.metric("Top Match Score", f"{float(score)*100:.2f}%")

    # Live Agent Feed
    st.markdown("### Live Agent Feed")
    if not feed:
        st.info("No feed yet.")
    else:
        for ev in feed[-120:]:
            st.write(f"**[{ev.get('layer')} {ev.get('agent')}]** {ev.get('message')}")

    # Audit Trail
    st.markdown("### Audit Trail (Steps)")
    with st.expander("Show steps"):
        for s in steps:
            st.write(f"- [{s.get('layer_id')}] {s.get('tool_name')} | {s.get('status')} | {s.get('step_id')}")

    # Ranking view
    st.markdown("### One-Click Ranking")
    ranking_ref = artifacts.get("ranking")
    ranking = None
    if ranking_ref and ranking_ref.get("path") and Path(ranking_ref["path"]).exists():
        ranking = json.loads(Path(ranking_ref["path"]).read_text(encoding="utf-8"))
        st.dataframe(
            [{
                "score_%": round(float(x.get("overall_match_percent",0.0)), 2),
                "role": x.get("role_title"),
                "company": x.get("company"),
                "source": x.get("source"),
                "url": x.get("url"),
                "missing_required": ", ".join((x.get("missing_required_skills") or [])[:5]),
            } for x in ranking],
            use_container_width=True
        )
    else:
        st.caption("Ranking not available yet.")

    # HITL controls
    st.markdown("### Human-in-the-Loop Controls")
    if status == "needs_human_approval" and pending == "review_ranking":
        c1, c2 = st.columns(2)
        with c1:
            if st.button("✅ Approve Ranking → Generate Drafts", type="primary", use_container_width=True):
                rr = requests.post(f"{API_BASE}/action/{run_id}", json={"action_type": "approve_ranking", "payload": {}}, timeout=30)
                st.success("Approved. Draft generation started.")
        with c2:
            reason = st.text_input("Reason to refine ranking (optional)", value="")
            if st.button("❌ Reject Ranking → Re-run Discovery", use_container_width=True):
                rr = requests.post(f"{API_BASE}/action/{run_id}", json={"action_type": "reject_ranking", "payload": {"reason": reason}}, timeout=30)
                st.warning("Rejected. Discovery refinement started.")

    if status == "needs_human_approval" and pending == "review_drafts":
        c1, c2 = st.columns(2)
        with c1:
            if st.button("✅ Approve Drafts → Submit (Simulated)", type="primary", use_container_width=True):
                rr = requests.post(f"{API_BASE}/action/{run_id}", json={"action_type": "approve_drafts", "payload": {}}, timeout=30)
                st.success("Approved. Submission started.")
        with c2:
            reason = st.text_input("Reason to revise drafts (optional)", value="")
            if st.button("❌ Reject Drafts → Back to Ranking", use_container_width=True):
                rr = requests.post(f"{API_BASE}/action/{run_id}", json={"action_type": "reject_drafts", "payload": {"reason": reason}}, timeout=30)
                st.warning("Rejected drafts. Returning to ranking review.")

    # Drafts bundle preview
    st.markdown("### Drafts + Strategies")
    drafts_ref = artifacts.get("drafts_bundle")
    if drafts_ref and drafts_ref.get("path") and Path(drafts_ref["path"]).exists():
        bundle = json.loads(Path(drafts_ref["path"]).read_text(encoding="utf-8"))
        st.json(bundle)
    else:
        st.caption("Draft bundle not available yet.")

    # Downloads
    st.markdown("### Downloads")
    d1, d2 = st.columns(2)
    with d1:
        zip_ref = artifacts.get("career_dossier_zip")
        if zip_ref and zip_ref.get("path") and Path(zip_ref["path"]).exists():
            p = Path(zip_ref["path"])
            st.download_button("Download Career Dossier (ZIP)", data=p.read_bytes(), file_name=p.name, mime="application/zip")
        else:
            st.caption("Dossier ZIP not ready.")
    with d2:
        pdf_ref = artifacts.get("xai_transparency_pdf")
        if pdf_ref and pdf_ref.get("path") and Path(pdf_ref["path"]).exists():
            p = Path(pdf_ref["path"])
            st.download_button("Download XAI Transparency (PDF)", data=p.read_bytes(), file_name=p.name, mime="application/pdf")
        else:
            st.caption("XAI PDF not ready.")

    with st.expander("Full State JSON"):
        st.json(data)
'''
safe_overwrite("app/ui/dashboard.py", DASHBOARD)

APP_MAIN = r'''
from __future__ import annotations
import sys
from pathlib import Path

# ensure src/ is on path for streamlit local runs
ROOT = Path(__file__).resolve().parents[1]
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from app.ui.dashboard import main

main()
'''
safe_overwrite("app/main.py", APP_MAIN)

RUN_APP = r'''
from __future__ import annotations

import os
import subprocess
import sys
import time
from pathlib import Path


def main() -> int:
    """
    Description: Launch FastAPI brain + Streamlit Mission Control locally (indestructible).
    Layer: L0
    Input: None
    Output: exit code
    """
    root = Path(__file__).resolve().parent
    env = os.environ.copy()
    env["PYTHONPATH"] = str(root / "src") + (os.pathsep + env.get("PYTHONPATH", "") if env.get("PYTHONPATH") else "")

    uvicorn_cmd = [sys.executable, "-m", "uvicorn", "careeragent.api.main:app", "--host", "127.0.0.1", "--port", "8000", "--reload"]
    streamlit_cmd = [sys.executable, "-m", "streamlit", "run", "app/main.py", "--server.port", "8501"]

    print("Starting FastAPI:", " ".join(uvicorn_cmd))
    api = subprocess.Popen(uvicorn_cmd, env=env)

    time.sleep(1.2)

    print("Starting Streamlit:", " ".join(streamlit_cmd))
    ui = subprocess.Popen(streamlit_cmd, env=env)

    try:
        api.wait()
        ui.wait()
        return 0
    except KeyboardInterrupt:
        api.terminate()
        ui.terminate()
        return 130


if __name__ == "__main__":
    raise SystemExit(main())
'''
safe_overwrite("run_app.py", RUN_APP)

# Minimal dependency hint (not auto-editing pyproject here to avoid accidental toml corruption)
print("\nNEXT: Ensure deps installed:")
print("uv add fastapi uvicorn streamlit reportlab twilio pydantic-settings httpx pypdf")
print("uv sync")
print("\nThen run:")
print("uv run python run_app.py")


BACKUP: src/careeragent/config.py.bak_20260220_203044
WROTE: src/careeragent/config.py
BACKUP: src/careeragent/services/db_service.py.bak_20260220_203044
WROTE: src/careeragent/services/db_service.py
BACKUP: src/careeragent/services/notification_service.py.bak_20260220_203044
WROTE: src/careeragent/services/notification_service.py
BACKUP: src/careeragent/orchestration.py.bak_20260220_203044
WROTE: src/careeragent/orchestration.py
BACKUP: src/careeragent/api/main.py.bak_20260220_203044
WROTE: src/careeragent/api/main.py
BACKUP: app/ui/dashboard.py.bak_20260220_203044
WROTE: app/ui/dashboard.py
BACKUP: app/main.py.bak_20260220_203044
WROTE: app/main.py
BACKUP: run_app.py.bak_20260220_203044
WROTE: run_app.py

NEXT: Ensure deps installed:
uv add fastapi uvicorn streamlit reportlab twilio pydantic-settings httpx pypdf
uv sync

Then run:
uv run python run_app.py
